<a href="https://colab.research.google.com/github/lukeschaller/Take_a_Byte/blob/main/food_segmenter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow.compat.v2 as tf
import tensorflow_hub as hub
import numpy as np
from PIL import Image

In [ ]:
categories = ["background", "vegetables | leafy_greens", "vegetables | stem_vegetables", "vegetables | non-starchy_roots", "vegetables | other", "fruits", "protein | meat", "protein | poultry", "protein | seafood", "protein | eggs", "protein | beans/nuts", "starches/grains | baked_goods", "starches/grains | rice/grains/cereals", "starches/grains | noodles/pasta", "starches/grains | starchy_vegetables", "starches/grains | other", "soups/stews", "herbs/spices", "dariy", "snacks", "sweets/desserts", "beverages", "fats/oils/sauces", "food_containers", "dining_tools", "other_food"]

In [ ]:
model_path = "/content/Model1"
model = tf.saved_model.load(model_path)

In [ ]:
signature = model.signatures['default']

In [ ]:
# Function to process the input image
def ProcessImage(filename):
  image = Image.open(filename)
  image = image.resize((513, 513))  # Resize to match the model's expected input size
  image = np.array(image)  # Convert to NumPy array
  image = image / 255.0  # Normalize pixel values to [0, 1]

  image_with_batch = np.expand_dims(image, axis=0)
  return image_with_batch

In [ ]:
# Make predictions using the specified signature.
image_path = '/content/pizza.png'

predictions = signature(tf.constant(ProcessImage(image_path), dtype=tf.float32))

In [ ]:
# Get both outputs (%'s and model predictions)
semantic_probabilities = predictions['food_group_segmenter:semantic_probabilities'].numpy()
semantic_predictions = predictions['food_group_segmenter:semantic_predictions'].numpy()

In [ ]:
final_output = {}
# Loop through categories
for i in range(len(categories)):
  # Get the number of occurences of that category index in the prediction output
  category_count = np.count_nonzero(semantic_predictions == i)

  # Get the % of each category
  category_percentage = round(((category_count / semantic_predictions.size) * 100), 2)

  final_output[categories[i]] = str(category_percentage) + " %"

  # Print out category
  print(f'{categories[i]} : {category_percentage}%')



background : 45.08%
vegetables | leafy_greens : 0.0%
vegetables | stem_vegetables : 0.0%
vegetables | non-starchy_roots : 0.0%
vegetables | other : 2.68%
fruits : 0.12%
protein | meat : 0.04%
protein | poultry : 0.04%
protein | seafood : 0.0%
protein | eggs : 0.0%
protein | beans/nuts : 0.0%
starches/grains | baked_goods : 0.0%
starches/grains | rice/grains/cereals : 0.0%
starches/grains | noodles/pasta : 0.01%
starches/grains | starchy_vegetables : 0.24%
starches/grains | other : 2.52%
soups/stews : 0.0%
herbs/spices : 7.88%
dariy : 32.7%
snacks : 0.0%
sweets/desserts : 0.0%
beverages : 0.0%
fats/oils/sauces : 0.0%
food_containers : 7.04%
dining_tools : 1.65%
other_food : 0.0%
